# Syqure Python Demo

This notebook demonstrates using the syqure Python library to compile and run Codon/Sequre programs.

## Install Syqure Python Package

First, build and install the Python package:

In [1]:
!cd .. && ./install-python.sh

🔨 Building syqure Python package...
🔨 Building syqure Python package...
📦 Building wheel...
🍹 Building a mixed python/rust project
🔗 Found pyo3 bindings with abi3 support
🐍 Found CPython 3.13 at /Users/madhavajay/dev/syqure/workspace1/.venv/bin/python
📡 Using build options features from pyproject.toml
💻 Using `MACOSX_DEPLOYMENT_TARGET=11.0` for aarch64-apple-darwin by default
   Compiling syqure-py v0.1.0 (/Users/madhavajay/dev/syqure/workspace1/python)
    Finished ]8;;https://doc.rust-lang.org/cargo/reference/profiles.html#default-profiles\`release` profile [optimized]]8;;\ target(s) in 0.75s
📦 Built wheel for abi3 Python ≥ 3.8 to /Users/madhavajay/dev/syqure/workspace1/target/wheels/syqure-0.1.0-cp38-abi3-macosx_11_0_arm64.whl
✅ Build complete!

📦 Wheel location: target/wheels/
-rw-r--r--@ 1 madhavajay  staff   302K 27 Nov 16:11 target/wheels/syqure-0.1.0-cp38-abi3-macosx_11_0_arm64.whl

To install locally:
  pip install target/wheels/syqure-*.whl

📦 Installing syqure...
Using u

## Import and Check Version

In [2]:
# Workaround for directory name conflict - change to temp directory
import os
import sys

# Save original directory
orig_dir = os.getcwd()
os.chdir('/tmp')

# Import syqure (now won't conflict with workspace syqure/ directory)
sys.path.insert(0, os.path.join(orig_dir, '.venv/lib/python3.13/site-packages'))
import syqure

# Return to original directory
os.chdir(orig_dir)

print(f"Syqure version: {syqure.__version__}")
print(f"Documentation: {syqure.__doc__}")

ImportError: dlopen(/Users/madhavajay/dev/syqure/workspace1/.venv/lib/python3.13/site-packages/syqure/syqure.abi3.so, 0x0002): Library not loaded: @rpath/libcodonrt.dylib
  Referenced from: <1AAB8136-E246-303C-BACB-DA19F19994A0> /Users/madhavajay/dev/syqure/workspace1/.venv/lib/python3.13/site-packages/syqure/syqure.abi3.so
  Reason: tried: '/Users/madhavajay/dev/syqure/workspace1/.venv/lib/python3.13/site-packages/syqure/lib/codon/libcodonrt.dylib' (no such file), '$ORIGIN/lib/codon/libcodonrt.dylib' (no such file), '/Users/madhavajay/dev/syqure/workspace1/.venv/lib/python3.13/site-packages/syqure/lib/codon/libcodonrt.dylib' (no such file), '$ORIGIN/lib/codon/libcodonrt.dylib' (no such file)

## Compile and Run Example

Run the two-party sum example (equivalent to `cargo run -p syqure -- example/two_party_sum_simple.codon`):

In [ ]:
import os

# Set up options with correct codon path
opts = syqure.CompileOptions(
    codon_path=os.path.abspath("../codon/install"),
    plugin="sequre",
    run_after_build=True
)

# Simple usage - compile and run
compiler = syqure.Syqure(opts)
compiler.compile_and_run("../example/two_party_sum_simple.codon")

## Advanced Usage with Custom Options

In [ ]:
import os

# Create custom compile options with absolute path
opts = syqure.CompileOptions(
    codon_path=os.path.abspath("../codon/install"),
    plugin="sequre",
    release=False,
    run_after_build=True
)

# Create compiler instance
compiler = syqure.Syqure(opts)

# Compile and run
compiler.compile_and_run("../example/two_party_sum_simple.codon")

## Build Only (No Run)

In [ ]:
assert False

In [ ]:
import os

# Build without running
opts = syqure.CompilaeOptions(
    codon_path=os.path.abspath("../codon/install"),
    plugin="sequre",
    run_after_build=False
)

compiler = syqure.Syqure(opts)
output_path = compiler.compile("../example/two_party_sum_simple.codon")

if output_path:
    print(f"✅ Compiled to: {output_path}")
    print(f"Run with: {output_path}")

## Using the Default Compiler

In [ ]:
import os

# Note: Default uses CODON_PATH env var or "codon/install"
# For this demo, explicitly set the path
opts = syqure.CompileOptions(codon_path=os.path.abspath("../codon/install"))
compiler = syqure.Syqure(opts)
compiler.compile_and_run("../example/two_party_sum_simple.codon")